## Æther Composition Priority
| z-index | Group | Item(s) |
| --- | --- | --- |
| 0 | Background | Background color |
| 1 | Background | Tilemap |
| 2 | Interaction | Non-player sprites |
| 3 | Interaction | Player sprite |
| 4 | Interaction | Projectiles |
| 5 | Effects | Visual effects |
| 6 | Overlay | UI |


## Steps:

### Layer 0: Background 
---
- **Goal: Print a solid color background to the terminal display.**
  - [x] Print a single color to the terminal.
  - [x] Fills the terminal by default.
  - [ ] ~~Never exceeds terminal bounds~~
  - [x] Can be resized smaller than the terminal
  - [x] Use 256-color extended ANSI codes.
  - [x] Keep a `term_utils.py` module for common terminal operations.
- **Classes Needed**
  - [x] NyxEntity
  - [x] NyxEntityManager
  - [x] NyxComponentStore
  - [x] NyxComponent (ABC)
  - [x] BackgroundColorComponent
  - [x] NyxSystem (ABC)
  - [x] RendererSystem
  - [x] AetherCompositor
  - [x] HemeraRenderer

- After a few trials, working up through the composition layers seems the best way to approach the problem. 
- So far so good through the `NyxSystem` class. Next is working on rendering collection.
- Render system collects and sends the prioritized entity dict to Aether
- Basic HemeraRenderer created, rendering a pixel to the screen

Next: Aether -> Hemera

### Layer 1: Tilemap 
---
- **Goal**
  - [x] Print an array of tiles to the terminal
  - [x] Fill zeros with bg color
  - [x] Cache/buffer tilemap after compute
  - [x] Store tilemaps in a resource or store by ID
- **Classes Needed**
  - [x] TilesetStore
  - [x] TilemapComponent
  - [x] Tilemap
  - [x] TilemapSystem
  - [x] SceneComponent
  - [x] PositionComponent

- Decided to move `Store`-type dictionaries to class attributes for easy access. `TilesetStore` has been made with class attributes.
- SceneComponent allows for a global container/flag
- PositionComponent should allow for scrolling

### Intermediate Goal: Subpixel Rendering
---
- **Goal**
  - [x] Update HemeraTermFx to use subpixel drawing
  - [x] Adjust the current calculations to match the doubled height resolution.
  - [x] Print 256-color tilemap in subpixels to the terminal. 

- Used simple list iterations for the actual printing, for now.
- Explore if its easier or faster to slice the ndarray before iterating.

In [ ]:
import numpy as np

new_subpixel_frame = np.ones((10, 10), dtype=np.uint8)
old_frame = np.eye(10, dtype=np.uint8)
delta_buffer = np.where(new_subpixel_frame != old_frame, new_subpixel_frame, 0)

print(old_frame)
print(delta_buffer)


[[1 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 1]]
[[0 1 1 1 1 1 1 1 1 1]
 [1 0 1 1 1 1 1 1 1 1]
 [1 1 0 1 1 1 1 1 1 1]
 [1 1 1 0 1 1 1 1 1 1]
 [1 1 1 1 0 1 1 1 1 1]
 [1 1 1 1 1 0 1 1 1 1]
 [1 1 1 1 1 1 0 1 1 1]
 [1 1 1 1 1 1 1 0 1 1]
 [1 1 1 1 1 1 1 1 0 1]
 [1 1 1 1 1 1 1 1 1 0]]


In [93]:
# Subpixel and Delta testing
import numpy as np

old_frame = np.array([
    [0 for _ in range(5)],  # fg color
    [5 for _ in range(5)],  # bg color
    [11 for _ in range(5)], # fg color
    [16 for _ in range(5)], # bg color
    [22 for _ in range(5)], # fg color
    [27 for _ in range(5)], # bg color
    [33 for _ in range(5)], # fg color
    [38 for _ in range(5)], # bg color
    [44 for _ in range(5)], # fg color
    [49 for _ in range(5)], # bg color
], dtype=np.uint8)

new_frame = np.arange(50).reshape(10, 5) # Mock frame


old_subpixel_frame = np.stack([old_frame[::2, :], old_frame[1::2, :]], axis=1) # split fg and bg to make 3d array
new_subpixel_frame = np.stack([new_frame[::2, :], new_frame[1::2, :]], axis=1) # split fg and bg to make 3d array

# Keep new_subpixel_frame fg & bg pixel at a given x, y index if EITHER fg OR bg color had changed from the same z-pair at the same x, y coordinates in the old_subpixel_frame
# If both pairs are the same, replace with zeros

delta_frame = np.where(
    np.any(new_subpixel_frame != old_subpixel_frame, axis=1, keepdims=True),
    new_subpixel_frame,
    np.zeros_like(new_subpixel_frame)
)
print(delta_subpixel_frame)


# Mock slice:
old_frame_pair = np.array([16, 17])

# Get subpixel pair = ([[16, 17]])
new_frame_pair = new_subpixel_frame[1:2:, 1:2:, 1:3] 

# Compare. Returns True if ANY change
changed = np.any(new_frame_pair != old_frame_pair)
print(f"Changed = {changed}")

[[[ 0  1  2  3  4]
  [10  0 12 13 14]
  [20 21  0 23 24]
  [30 31 32  0 34]
  [40 41 42 43  0]]

 [[ 0  6  7  8  9]
  [15  0 17 18 19]
  [25 26  0 28 29]
  [35 36 37  0 39]
  [45 46 47 48  0]]]
Changed = False
